In [1]:
import ugradio
import numpy as np
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
import dish
import cartopy.crs as ccrs
from scipy.interpolate import interp2d
from tqdm import tqdm, trange
%matplotlib inline
from astropy.coordinates import SkyCoord
from astropy import constants as c
import astropy.units as u
from ugradio.leo import *

No module named 'rtlsdr'


In [2]:
def calibrate(data_nd0, data_nd1, pol):
    diff = np.array(data_nd1) - np.array(data_nd0)
    if pol == 0:
        T = 90
    elif pol == 1:
        T = 65
    else: print('Bruh its literally either a 0 or 1 you cant fuck it up')
    G = T/diff
    t_line = G * data_nd0
    return t_line

def calibrate1(data_nd0, data_nd1, pol):
    diff = np.median(np.array(data_nd1) - np.array(data_nd0))
    if pol == 0:
        T = 90
    elif pol == 1:
        T = 65
    else: print('Bruh its literally either a 0 or 1 you cant fuck it up')
    G = T/diff
    t_line = G * data_nd0
    return t_line

def calibrate2(data_nd0, data_nd1, pol):
    
    F_low = 146.5
    F_high = 148.5
    freq = np.linspace(144,156,8192)
    condition = (freq >= F_low) * (freq <= F_high)
    trunc_freq = freq[condition]
    
    diff = np.median(np.array(data_nd1[:,condition]) - np.array(data_nd0[:,condition]))
    
    if pol == 0:
        T = 90
    elif pol == 1:
        T = 65
    else: print('Bruh its literally either a 0 or 1 you cant fuck it up')
        
    G = T/diff
    t_line = G * data_nd0
    return t_line

In [3]:
nfiles = 697
nfreq = 8192

In [4]:
nspec = 50

list_0 = np.empty((nfiles, nspec, nfreq))
# list_1 = np.empty((nfiles, nspec, nfreq))

for i in trange(nfiles):
    
    f = fits.open(f'lab4_data/{i}_nd0.fits')
    
    for j in range(nspec):
        
        list_0[i,j] = f[j+1].data['auto0_real']
        
#         list_1[i,j] = f[j+1].data['auto1_real']

100%|██████████| 697/697 [02:24<00:00,  4.82it/s]


In [5]:
noise_spec = 5

noise_0 = np.empty((nfiles, noise_spec, nfreq))
# noise_1 = np.empty((nfiles, noise_spec, nfreq))

for i in trange(nfiles):
    
    f = fits.open(f'lab4_data/{i}_nd1.fits')
    
    for j in range(noise_spec):
        
        noise_0[i,j] = f[j+1].data['auto0_real']
        
#         noise_1[i,j] = f[j+1].data['auto1_real']

100%|██████████| 697/697 [00:15<00:00, 45.52it/s]


In [ ]:
median_specs_0 = np.median(list_0, axis=1)
# median_specs_1 = np.median(list_1, axis=1)

In [ ]:
median_noise_0 = np.median(noise_0, axis=1)
# median_noise_1 = np.median(noise_1, axis=1)

In [ ]:
Tline_0 = calibrate2(median_specs_0, median_noise_0, 0)
# Tline_1 = calibrate2(median_specs_1, median_noise_1, 1)

In [ ]:
F_low = 149.1
F_high = 151.1
freq = np.linspace(144,156,8192)
condition = (freq >= F_low) * (freq <= F_high)

ctline_0 = np.empty((nfiles, nfreq))
# ctline_1 = np.empty((nfiles, nfreq))

for i in trange(nfiles):
    sub_0 = Tline_0[i] - np.median(Tline_0[i,condition])
    ctline_0[i] = sub_0
#     sub_1 = Tline_1[i] - np.median(Tline_1[i,condition])
#     ctline_1[i] = sub_1

In [ ]:
trunc_freq = freq[condition]
trunc_spec = ctline_0[650, condition]

gaussian_fit, cov = ugradio.gauss.gaussfit(x = trunc_freq, y = trunc_spec,
                                          amp = trunc_spec.max(),
                                          avg = trunc_freq[np.argmax(trunc_spec)],
                                          sig = 0.1,
                                          return_cov = True)
gaussian_fit

In [ ]:
gaussian_val = ugradio.gauss.gaussval(trunc_freq, 
                                     gaussian_fit['amp'],
                                     gaussian_fit['avg'],
                                     gaussian_fit['sig'])

In [ ]:
plt.plot(trunc_freq,trunc_spec)
plt.plot(trunc_freq, gaussian_val)
plt.grid()
plt.show

In [ ]:
coords = np.array(dish.bubble_coords())

In [ ]:
ν_0 = 150.4

bubble_T = []
bubble_v = []
# bubble_vdopp = []
# bubble_vcorr =[]
bubble_σ = []

bubble_err = []

bubble_RA = []
bubble_DEC = []
bubble_JD = []

for i in trange(len(coords)):
    
    trunc_freq = freq[condition]
    trunc_spec = ctline_0[i, condition]
    
    gaussian_fit, cov = ugradio.gauss.gaussfit(x = trunc_freq, y = trunc_spec,
                                          amp = trunc_spec.max(),
                                          avg = trunc_freq[np.argmax(trunc_spec)],
                                          sig = 0.1,
                                          return_cov = True)
    
    f = fits.open(f'lab4_data/{i}_nd0.fits')
    
    RA = dict(f[0].header)['RA']
    bubble_RA.append(RA)
    
    DEC = dict(f[0].header)['DEC']
    bubble_DEC.append(DEC)
    
    JD = ugradio.timing.julian_date(dict(f[0].header)['JD'])
    bubble_JD.append(JD)
    
    T = gaussian_fit["amp"][0]
    bubble_T.append(T)
    
    v_avg = (gaussian_fit["avg"][0]-ν_0)/ν_0 * c.c.value * 10**-3
    bubble_v.append(v_avg)
    
#     v_dopp = (ugradio.doppler.get_projected_velocity(RA, DEC, JD, lat, lon, alt)).to(u.km/u.s).value
#     bubble_vdopp.append(v_dopp)
    
#     v_corr = v_avg + v_dopp
#     bubble_vcorr.append(v_corr)
    
    σ = np.abs((gaussian_fit["sig"][0])/ν_0 * c.c.value * 10**-3)
    bubble_σ.append(σ)
    
    err = np.sqrt(np.diag(cov))
    bubble_err.append(err)

In [ ]:
T_err = np.array(bubble_err)[:, 0]
v_err = np.array(bubble_err)[:, 1]
σ_err = np.array(bubble_err)[:, 2]

l = coords[:,0]
b = coords[:,1]

In [ ]:
df = pd.DataFrame({'T': bubble_T, 'v': bubble_v, 'σ': bubble_σ, 'T_err': T_err, 'v_err': v_err, 'σ_err': σ_err, 'l': l, 'b': b,
                  'RA':bubble_RA, 'DEC':bubble_DEC, 'JD':bubble_JD})
df

In [ ]:
df.to_csv('final_data')

In [18]:
f = fits.open(f'lab4_data/0_nd0.fits')
JD = (dict(f[0].header)['JD'])
RA = dict(f[0].header)['RA']
DEC = dict(f[0].header)['DEC']
#ugradio.doppler.get_projected_velocity(RA, DEC, JD, lat, lon, alt)
JD

2459692.297407407

In [16]:
ugradio.doppler.get_projected_velocity??

In [17]:
ugradio.timing.utc(JD)

'Thu Jan 29 11:14:52 1970'

In [32]:
t = ugradio.timing.julian_date()
ugradio.doppler.get_projected_velocity(ra=RA,dec=DEC,jd=JD)

ValueError: Column year failed to convert: invalid literal for int() with base 10: '<!'

In [26]:
JD

2459692.297407407